# Scrapper for the studies calendar (ics file) RTU

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from icalendar import Calendar, Event, vDatetime, vText
from datetime import datetime
import arrow, uuid, pytz, time

## Selecting right course

In [3]:
driver = webdriver.Chrome()
driver.get('https://nodarbibas.rtu.lv')

study_period_select = Select(driver.find_element(By.ID, 'semester-id'))
study_period_select.select_by_visible_text('2023/2024 Rudens semestris (23/24-R)') # Jūsu studiju periods

time.sleep(2)

faculty_program_select = Select(driver.find_element(By.ID, 'program-id'))
faculty_program_select.select_by_visible_text('Datorsistēmas (RDBD0)') # Jūsu fakultāte un programma

time.sleep(2)

course_select = Select(driver.find_element(By.ID, 'course-id'))
course_select.select_by_index(3) # Jūsu kurss (1. kurss ir 1, 2. kurss ir 2, utt.)

group_select = Select(driver.find_element(By.ID, 'group-id'))
group_select.select_by_index(2) # Jūsu grupa (atveriet lapu un paskatieties, kāds ir Jūsu grupas indekss)


## Select first 14 days of our calendar with date time and title

In [4]:
# Cerēsimm ka viņi neaminīs elementus, jo tad šis viss nestrādās
day_cells = driver.find_elements(By.CLASS_NAME, "fc-daygrid-day")

events_dict = {}

days_with_classes_counted = 0

for day_cell in day_cells:
    if days_with_classes_counted >= 14:
        break

    date_str = day_cell.get_attribute('data-date')
    
    if date_str:
        events_list = []
        
        events = day_cell.find_elements(By.CLASS_NAME, "fc-daygrid-event")
        days_with_classes_counted += 1

        if events:

            for event in events:
                event_time_str = event.find_element(By.CLASS_NAME, "fc-event-time").text
                start_time, end_time = event_time_str.split(" - ")
                event_title = event.find_element(By.CLASS_NAME, "fc-event-title").text
                
                events_list.append({
                    'start_time': f"{start_time}",
                    'end_time': f"{end_time}",
                    'title': event_title
                })
            
            events_dict[date_str] = events_list


In [5]:
driver.close()

## Creating ics file

In [11]:
cal = Calendar()
cal.add('version', '2.0')
cal.add('prodid', '-//RTU//STUDIES//1')

end_repeat_date = arrow.get("2023-12-24", "YYYY-MM-DD")

riga_tz = pytz.timezone('Europe/Riga')

for date_str, events in events_dict.items():
    for event_info in events:
        e = Event()
        e.add('uid', str(uuid.uuid4())) 
        e.add('dtstamp', datetime.now(tz=riga_tz))

        start_date_time = arrow.get(f"{date_str} {event_info['start_time']}", "YYYY-MM-DD HH:mm").replace(tzinfo=riga_tz).datetime
        end_date_time = arrow.get(f"{date_str} {event_info['end_time']}", "YYYY-MM-DD HH:mm").replace(tzinfo=riga_tz).datetime

        e.add('summary', event_info['title'])
        e.add('dtstart', start_date_time)
        e.add('dtend', end_date_time)
        e.add('rrule', {'freq': 'weekly', 'interval': 2, 'until': end_repeat_date.datetime})
        cal.add_component(e)

with open('studijas_rudens_23_24.ics', 'wb') as my_file:
    my_file.write(cal.to_ical())
